In [22]:
import os
import nbformat
from nbconvert import PythonExporter

# 定義需要檢查的旗標變數名稱
TRANSFORM_FLAG = "need_transform_to_py"

def convert_ipynb_to_py(folder_path):
    """
    遞迴遍歷指定資料夾，將所有 .ipynb 檔案轉換為 .py 檔案。
    僅當 Notebook 的第一個程式碼 Cell 包含變數 need_transform_to_py = True 時才執行轉換。
    """
    
    # 初始化 nbconvert 匯出器
    exporter = PythonExporter()
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".ipynb") and not file.startswith("."):
                ipynb_path = os.path.join(root, file)
                py_path = os.path.splitext(ipynb_path)[0] + ".py"

                try:
                    # 1. 讀取 .ipynb 檔案
                    with open(ipynb_path, "r", encoding="utf-8") as f:
                        notebook_node = nbformat.read(f, as_version=4)

                    # 2. 檢查轉換旗標 (TRANSFORM_FLAG)
                    should_convert = False
                    
                    # 尋找第一個程式碼 Cell
                    first_code_cell_source = ""
                    for cell in notebook_node.cells:
                        if cell.cell_type == 'code':
                            first_code_cell_source = cell.source
                            break
                    
                    # 檢查旗標是否存在並為 True
                    # 這裡使用簡單的字串檢查邏輯
                    if first_code_cell_source:
                        # 檢查變數宣告，例如: need_transform_to_py = True
                        if f"{TRANSFORM_FLAG}=True" in first_code_cell_source.replace(" ", ""):
                            should_convert = True

                    if not should_convert:
                        print(f"Skipping: {ipynb_path} (Flag '{TRANSFORM_FLAG} = True' not found in first code cell)")
                        continue

                    # 如果旗標為 True，則執行轉換
                    print(f"Converting: {ipynb_path} -> {py_path}")
                    
                    # 3. 轉換 .ipynb 到 .py
                    body, _ = exporter.from_notebook_node(notebook_node)

                    # 4. 過濾和清理程式碼 (移除 Jupyter 魔術指令)
                    lines = body.splitlines()
                    filtered_lines = []
                    for line in lines:
                        # 移除 Jupyter 的魔術指令及其轉換後的 Python 函式呼叫
                        if not line.strip().startswith("get_ipython().run_line_magic('load_ext', 'autoreload')") and \
                           not line.strip().startswith("get_ipython().run_line_magic('autoreload', '2')") and \
                           not line.strip().startswith("need_transform_to_py = True"):
                            filtered_lines.append(line)
                            
                    filtered_body = "\n".join(filtered_lines)
                        
                    # 5. 寫入 .py 檔案
                    with open(py_path, "w", encoding="utf-8") as f:
                        f.write(filtered_body)
                        print(f"Transformed.")
                
                except Exception as e:
                    print(f"⚠️ Conversion failed for {ipynb_path}: {e}")

    print("✅ All checked .ipynb files processed.")

if __name__ == "__main__":
    # 更改為你的資料夾名稱 (範例中使用相對於程式碼位置的上層資料夾)
    # 請根據您的實際專案結構調整路徑
    convert_ipynb_to_py("../application")

Converting: ../application\fetch_youtube_screenwork_comments.ipynb -> ../application\fetch_youtube_screenwork_comments.py
Transformed.
Converting: ../application\fetch_youtube_trending_by_keyword.ipynb -> ../application\fetch_youtube_trending_by_keyword.py
Transformed.
Skipping: ../application\regression_analysis.ipynb (Flag 'need_transform_to_py = True' not found in first code cell)
Skipping: ../application\trickery_drama_evolution_study.ipynb (Flag 'need_transform_to_py = True' not found in first code cell)
Skipping: ../application\trickery_drama_evolution_study_fetch_youtube_video.ipynb (Flag 'need_transform_to_py = True' not found in first code cell)
Converting: ../application\youtube_topic_modeling_pipeline.ipynb -> ../application\youtube_topic_modeling_pipeline.py
Transformed.
Converting: ../application\.ipynb_checkpoints\fetch_youtube_screenwork_comments-checkpoint.ipynb -> ../application\.ipynb_checkpoints\fetch_youtube_screenwork_comments-checkpoint.py
Transformed.
Converting: